In [50]:
using OhMyJulia
using DataFrames
using PyCall
@pyimport gurobipy as grb

include("path.jl")
include("weight.jl")
ENV["PATH"] *= ":/usr/local/gurobi/bin/:/home/zhangshiwei/.opam/4.06.0/bin"

data_list = ["abilene", "Cernet", "Cesnet201006", "Chinanet", "Esnet", "Garr200112", "Grnet", "Ibm", "jelly16", "Nordu1997", "Tinet"]
algo_list = ["Ecmp", "Edskp", "Ksp", "Mcf", "Raeck", "Vlb"]

11-element Array{String,1}:
 "abilene"     
 "Cernet"      
 "Cesnet201006"
 "Chinanet"    
 "Esnet"       
 "Garr200112"  
 "Grnet"       
 "Ibm"         
 "jelly16"     
 "Nordu1997"   
 "Tinet"       

In [35]:
function read_topo(file)
    nodes, edges = Set{String}(), String[]
    for line in eachline(file) @when (m = match(r"(s\d+) -> (s\d+)", line)) != nothing
        push!(nodes, m[1], m[2])
        push!(edges, "$(m[1]) $(m[2])")
    end
    nodes, edges
end

function read_demand(hosts, demands)
    results = []
    hosts = readlines(hosts)
    n = length(hosts)
    for line in eachline(demands)
        line = map(x->parse(f64, x), split(line, ' '))
        demand = Dict{String, Float64}()
        for i in 1:n, j in 1:n @when i != j
            demand["$(hosts[i]) $(hosts[j])"] = line[(i-1) * n + j] / 2^20 # to Megabyte
        end
        push!(results, demand)
    end
    results
end

read_demand (generic function with 2 methods)

In [56]:
function gen_gravity(data)
    hosts = readlines("data/$data.hosts")
    open("data/$data.demands", "w") do fout
        for l in 1:5
            α = 2^(20 + 2i)
            weigths = map(x->rand(), hosts)
            list = [ α * weigths[i] * weigths[j]
                     for i in 1:length(hosts)
                     for j in 1:length(hosts) ]
            println(fout, join(list, ' '))
        end
    end
end
for i in data_list
    gen_gravity(i)
end

In [65]:
for data in data_list
    nodes, edges = read_topo("data/$data.dot")
    hosts = readlines("data/$data.hosts")
    open("data/$data.budgets", "w") do fout
        for i in 1:length(hosts)
            println(fout, 15 * length(nodes))
        end
    end
end


In [26]:
data = "Ibm"

io, process = open(`yates -budget 16 $data`)
schemes = parse_schemes(io)

Dict{String,Dict{String,Array{Tuple{Array{String,1},Float64},1}}} with 6 entries:
  "Ecmp"  => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…
  "Vlb"   => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…
  "Mcf"   => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…
  "Ksp"   => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…
  "Edksp" => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…
  "Raeke" => Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1…

In [ ]:
for data in data_list
    io, process = open(`yates -budget 16 $data`)
    origin_schemes = parse_schemes(io)
    nodes, edges = read_topo("data/$data.dot")
    demands = read_demand("data/$data.hosts", "data/$data.demands")
    
    for algo in algo_list
        x = select_program(nodes, edges, schemes["Vlb"], 20length(nodes))
    end
end

In [58]:
x = select_program(nodes, edges, schemes["Vlb"], 20length(nodes))

Dict{String,Array{Tuple{Array{String,1},Float64},1}} with 306 entries:
  "h1 h14"  => Tuple{Array{String,1},Float64}[(String["h1 s1", "s1 s17", "s17 s…
  "h14 h13" => Tuple{Array{String,1},Float64}[(String["h14 s14", "s14 s13", "s1…
  "h3 h5"   => Tuple{Array{String,1},Float64}[(String["h3 s3", "s3 s2", "s2 s4"…
  "h9 h12"  => Tuple{Array{String,1},Float64}[(String["h9 s9", "s9 s17", "s17 s…
  "h12 h17" => Tuple{Array{String,1},Float64}[(String["h12 s12", "s12 s11", "s1…
  "h0 h17"  => Tuple{Array{String,1},Float64}[(String["h0 s0", "s0 s3", "s3 s2"…
  "h7 h17"  => Tuple{Array{String,1},Float64}[(String["h7 s7", "s7 s0", "s0 s3"…
  "h2 h13"  => Tuple{Array{String,1},Float64}[(String["h2 s2", "s2 s9", "s9 s17…
  "h8 h10"  => Tuple{Array{String,1},Float64}[(String["h8 s8", "s8 s12", "s12 s…
  "h7 h4"   => Tuple{Array{String,1},Float64}[(String["h7 s7", "s7 s0", "s0 s3"…
  "h8 h5"   => Tuple{Array{String,1},Float64}[(String["h8 s8", "s8 s4", "s4 s5"…
  "h5 h9"   => Tuple{Array{String,1},F

In [7]:
x = select_hard_nop(nodes, edges, schemes["Vlb"], 20length(nodes))

1

In [59]:
minimize_maximum_link_utilization(nodes, edges, demands[1], x)

(Dict("h1 h14"=>Tuple{Array{String,1},Float64}[(String["h1 s1", "s1 s17", "s17 s14", "s14 h14"], 0.0), (String["h1 s1", "s1 s5", "s5 s15", "s15 s14", "s14 h14"], 1.0), (String["h1 s1", "s1 s6", "s6 s16", "s16 s15", "s15 s14", "s14 h14"], 0.0), (String["h1 s1", "s1 s17", "s17 s11", "s11 s12", "s12 s13", "s13 s14", "s14 h14"], 0.0)],"h14 h13"=>Tuple{Array{String,1},Float64}[(String["h14 s14", "s14 s13", "s13 h13"], 1.0), (String["h14 s14", "s14 s17", "s17 s11", "s11 s12", "s12 s13", "s13 h13"], 0.0), (String["h14 s14", "s14 s15", "s15 s5", "s5 s4", "s4 s8", "s8 s12", "s12 s13", "s13 h13"], 0.0)],"h3 h5"=>Tuple{Array{String,1},Float64}[(String["h3 s3", "s3 s2", "s2 s4", "s4 s5", "s5 h5"], 0.0), (String["h3 s3", "s3 s0", "s0 s7", "s7 s6", "s6 s1", "s1 s5", "s5 h5"], 1.0), (String["h3 s3", "s3 s2", "s2 s9", "s9 s17", "s17 s1", "s1 s5", "s5 h5"], 0.0), (String["h3 s3", "s3 s0", "s0 s7", "s7 s6", "s6 s16", "s16 s15", "s15 s5", "s5 h5"], 0.0), (String["h3 s3", "s3 s2", "s2 s9", "s9 s17", "s17 

In [60]:
minimize_maximum_link_utilization_then_maximize_throuput(nodes, edges, demands[1], x) 

(Dict{Any,Any}(Pair{Any,Any}("h1 h14", Tuple{Array{String,1},Float64}[(String["h1 s1", "s1 s17", "s17 s14", "s14 h14"], 0.0), (String["h1 s1", "s1 s5", "s5 s15", "s15 s14", "s14 h14"], 1.0), (String["h1 s1", "s1 s6", "s6 s16", "s16 s15", "s15 s14", "s14 h14"], 0.0), (String["h1 s1", "s1 s17", "s17 s11", "s11 s12", "s12 s13", "s13 s14", "s14 h14"], 0.0)]),Pair{Any,Any}("h14 h13", Tuple{Array{String,1},Float64}[(String["h14 s14", "s14 s13", "s13 h13"], 1.0), (String["h14 s14", "s14 s17", "s17 s11", "s11 s12", "s12 s13", "s13 h13"], 0.0), (String["h14 s14", "s14 s15", "s15 s5", "s5 s4", "s4 s8", "s8 s12", "s12 s13", "s13 h13"], 0.0)]),Pair{Any,Any}("h3 h5", Tuple{Array{String,1},Float64}[(String["h3 s3", "s3 s2", "s2 s4", "s4 s5", "s5 h5"], 0.0), (String["h3 s3", "s3 s0", "s0 s7", "s7 s6", "s6 s1", "s1 s5", "s5 h5"], 1.0), (String["h3 s3", "s3 s2", "s2 s9", "s9 s17", "s17 s1", "s1 s5", "s5 h5"], 0.0), (String["h3 s3", "s3 s0", "s0 s7", "s7 s6", "s6 s16", "s16 s15", "s15 s5", "s5 h5"], 0.0

In [63]:
keys(schemes)

Base.KeyIterator for a Dict{String,Dict{String,Array{Tuple{Array{String,1},Float64},1}}} with 6 entries. Keys:
  "Ecmp"
  "Vlb"
  "Mcf"
  "Ksp"
  "Edksp"
  "Raeke"